# Training a sequence tagger for Slot Filling

<a target="_blank" href="https://www.recogn.ai/biome-text/documentation/tutorials/2-Training_a_sequence_tagger_for_Slot_Filling.html"><img class="icon" src="https://www.recogn.ai/biome-text/assets/img/biome-isotype.svg" width=24 /></a>
[View on recogn.ai](https://www.recogn.ai/biome-text/documentation/tutorials/2-Training_a_sequence_tagger_for_Slot_Filling.html)
    
<a target="_blank" href="https://colab.research.google.com/github/recognai/biome-text/blob/master/docs/docs/documentation/tutorials/2-Training_a_sequence_tagger_for_Slot_Filling.ipynb"><img class="icon" src="https://www.tensorflow.org/images/colab_logo_32px.png" width=24 /></a>
[Run in Google Colab](https://colab.research.google.com/github/recognai/biome-text/blob/master/docs/docs/documentation/tutorials/2-Training_a_sequence_tagger_for_Slot_Filling.ipynb)
        
<a target="_blank" href="https://github.com/recognai/biome-text/blob/master/docs/docs/documentation/tutorials/2-Training_a_sequence_tagger_for_Slot_Filling.ipynb"><img class="icon" src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png" width=24 /></a>
[View source on GitHub](https://github.com/recognai/biome-text/blob/master/docs/docs/documentation/tutorials/2-Training_a_sequence_tagger_for_Slot_Filling.ipynb)

In this tutorial we will train a sequence tagger for filling slots in spoken requests.
The goal is to look for specific pieces of information in the request and tag the corresponding tokens accordingly. 
The requests will include several intents, from getting weather information to adding a song to a playlist, each requiring its own set of slots.
Therefore, slot filling often goes hand in hand with intent classification.
In this tutorial, however, we will only focus on the slot filling task.

Slot filling is closely related to [Named-entity recognition (NER)](https://en.wikipedia.org/wiki/Named-entity_recognition) and the model of this tutorial can also be used to train a NER system.

In this tutorial we will use the [SNIPS data set](https://github.com/snipsco/nlu-benchmark/tree/master/2017-06-custom-intent-engines) adapted by [Su Zhu](https://github.com/sz128/slot_filling_and_intent_detection_of_SLU/tree/master/data/snips) and our simple [data preparation notebook](https://biome-tutorials-data.s3-eu-west-1.amazonaws.com/token_classifier/data_prep.ipynb).

When running this tutorial in Google Colab, make sure to install *biome.text* first:

In [ ]:
!pip install -U git+https://github.com/recognai/biome-text.git

Ignore warnings and don't forget to restart your runtime afterwards (*Runtime -> Restart runtime*).

## Explore the data

Let's take a look at the data before starting with the configuration of our pipeline.
For this we create a `DataSource` instance providing a path to our data.

In [ ]:
from biome.text.data import DataSource

In [ ]:
train_ds = DataSource(source="https://biome-tutorials-data.s3-eu-west-1.amazonaws.com/token_classifier/train.json")
train_ds.head()

As we can see we have two relevant columns for our task: *text* and *labels*. 
The *intent* column will be ignored in this tutorial. 
The *path* column is added automatically by the [DataSource](https://www.recogn.ai/biome-text/api/biome/text/data/datasource.html#datasource) class to keep track of the source file.

The input already comes pre-tokenized and each token in the *text* column has a label/tag in the *labels* column, this means that both list always have the same length.
The labels are given in the [BIO tagging scheme](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)), which is widely used in Slot Filling/NER systems.

When specifying the [TokenClassification](https://www.recogn.ai/biome-text/api/biome/text/modules/heads/token_classification.html#tokenclassification) head (see [below](#Configure-your-biome.text-Pipeline)), the tokenization step in the pipeline is automatically disabled and the input is expected to be a list of tokens.

The [DataSource](https://www.recogn.ai/biome-text/api/biome/text/data/datasource.html#datasource) class stores the data in an underlying [Dask DataFrame](https://docs.dask.org/en/latest/dataframe.html) that you can easily access.
For example, let's check the size of our training data:

In [ ]:
len(train_ds.to_dataframe())

Or let's check how many different labels/tags we have:

In [ ]:
df = train_ds.to_dataframe().compute()
labels_total = df.labels.sum()
len(set(labels_total))

and how they are distributed:

In [ ]:
import pandas as pd
pd.Series(labels_total).value_counts()

::: tip Tip

The [TaskHead](https://www.recogn.ai/biome-text/api/biome/text/modules/heads/task_head.html#taskhead) of our model (the [TokenClassification](https://www.recogn.ai/biome-text/api/biome/text/modules/heads/token_classification.html#tokenclassification)) will expect a *text* and a *labels* column to be present in the dataframe. 
Since they are already present, there is no need for a `mapping` in the [DataSource](https://www.recogn.ai/biome-text/api/biome/text/data/datasource.html#datasource).

:::

## Configure your *biome.text* Pipeline

A typical [Pipeline](https://www.recogn.ai/biome-text/api/biome/text/pipeline.html#pipeline) consists of tokenizing the input, extracting features, applying a language encoding (optionally) and executing a task-specific head in the end.
After training a pipeline, you can use it to make predictions or explore the underlying model via the explore UI.

A *biome.text* pipeline has the following main components:

```yaml
name: # a descriptive name of your pipeline

tokenizer: # how to tokenize the input

features: # input features of the model

encoder: # the language encoder

head: # your task configuration

```

See the [Configuration section](https://www.recogn.ai/biome-text/documentation/user-guides/2-configuration.html) for a detailed description of how these main components can be configured.

In this tutorial we will create a [PipelineConfiguration](https://www.recogn.ai/biome-text/api/biome/text/configuration.html#pipelineconfiguration) programmatically, and use it to initialize the [Pipeline](https://www.recogn.ai/biome-text/api/biome/text/pipeline.html#pipeline).
You can also create your pipelines by providing a [python dictionary](https://www.recogn.ai/biome-text/api/biome/text/pipeline.html#from-config) (see the text classification [tutorial](https://www.recogn.ai/biome-text/documentation/tutorials/1-Training_a_text_classifier.html)), a YAML [configuration file](https://www.recogn.ai/biome-text/api/biome/text/pipeline.html#from-yaml) or a [pretrained model](https://www.recogn.ai/biome-text/api/biome/text/pipeline.html#from-pretrained).

A pipeline configuration is composed of several other [configuration classes](https://www.recogn.ai/biome-text/api/biome/text/configuration.html#biome-text-configuration), each one corresponding to one of the main components.

### Features

Let us first configure the features of our pipeline.
For our `word` feature we will use pretrained embeddings from [fasttext](https://fasttext.cc/docs/en/english-vectors.html), and our `char` feature will use the last hidden state of a [GRU](https://en.wikipedia.org/wiki/Gated_recurrent_unit) encoder to represent a word based on its characters.
Keep in mind that the `embedding_dim` parameter for the `word` feature must be equal to the dimensions of the pretrained embeddings!

::: tip Tip

If you do not provide any feature configurations, we will choose a very basic `word` feature by default.

:::

In [ ]:
from biome.text.configuration import FeaturesConfiguration, WordFeatures, CharFeatures

word_feature = WordFeatures(
    embedding_dim=300,
    weights_file="https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip",
)

char_feature = CharFeatures(
    embedding_dim=32,
    encoder={
        "type": "gru",
        "bidirectional": True,
        "num_layers": 1,
        "hidden_size": 32,
    },
    dropout=0.1
)

features_config = FeaturesConfiguration(
    word=word_feature, 
    char=char_feature
)

### Encoder

Next we will configure our encoder that takes as input a sequence of embedded word vectors and returns a sequence of encoded word vectors.
For this encoding we will use another larger GRU:

In [ ]:
from biome.text.modules.configuration import Seq2SeqEncoderConfiguration

encoder_config = Seq2SeqEncoderConfiguration(
    type="gru",
    bidirectional=True,
    num_layers=1,
    hidden_size=128,
)

### Head

The final configuration belongs to our [TaskHead](https://www.recogn.ai/biome-text/api/biome/text/modules/heads/task_head.html#taskhead).
It reflects the task our problem belongs to and can be easily exchanged with other types of heads keeping the same features and encoder.

::: tip Tip

Exchanging the heads you can easily pretrain a model on a certain task, such as [language modelling](https://www.recogn.ai/biome-text/api/biome/text/modules/heads/language_modelling.html#languagemodelling), and use its pretrained features and encoder for training the model on another task.

:::

For our task we will use a [TokenClassification](https://www.recogn.ai/biome-text/api/biome/text/modules/heads/token_classification.html#tokenclassification) head that allows us to tag each token individually:

In [ ]:
from biome.text.modules.heads import TokenClassificationConfiguration

labels = list(set([label[2:] for label in labels_total if label != 'O']))

head_config = TokenClassificationConfiguration(
    labels=labels,
    label_encoding="BIO",
    feedforward={
        "num_layers": 1,
        "hidden_dims": [128],
        "activations": ["relu"],
        "dropout": [0.1],
    },
)

### Pipeline

Now we can create a [PipelineConfiguration](https://www.recogn.ai/biome-text/api/biome/text/configuration.html#pipelineconfiguration) and finally initialize our [Pipeline](https://www.recogn.ai/biome-text/api/biome/text/pipeline.html#pipeline).

In [ ]:
from biome.text.configuration import PipelineConfiguration

pipeline_config = PipelineConfiguration(
    name="slot_filling_tutorial",
    features=features_config,
    encoder=encoder_config,
    head=head_config,
)

In [ ]:
from biome.text import Pipeline

pl = Pipeline.from_config(pipeline_config)

## Create a vocabulary

Before we can start the training we need to create the vocabulary for our model.
For this we define a `VocabularyConfiguration`.

In [ ]:
from biome.text import VocabularyConfiguration

Since we use pretrained word embeddings we will also consider the validation data when creating the vocabulary. 

In [ ]:
valid_ds = DataSource(source="https://biome-tutorials-data.s3-eu-west-1.amazonaws.com/token_classifier/valid.json")

We also get rid of the rarest words by adding the `min_count` argument and set it to 2 for the word feature vocabulary.
For a complete list of available arguments see the [VocabularyConfiguration API](https://www.recogn.ai/biome-text/api/biome/text/configuration.html#vocabularyconfiguration).

In [ ]:
vocab_config = VocabularyConfiguration(
    sources=[train_ds, valid_ds],
    min_count={"word": 2},
)

We then pass this configuration to our `Pipeline` to create the vocabulary.
Apart from the loading bar of building the vocabulary, there will be two more loading bars corresponding to the `weights_file` provided in the word feature:
 - the progress of downloading the file (this file will be cached)
 - the progress loading the weights from the file 

In [ ]:
pl.create_vocabulary(vocab_config)

After creating the vovocab_configbulary we can check the size of our entire model in terms of trainable parameters:

In [ ]:
pl.num_trainable_parameters

## Train your model

Now we have everything ready to start the training of our model:
- training data set
- vocabulary

As `trainer` we will use the default configuration that has sensible values and works alright for our experiment.
[This tutorial](https://www.recogn.ai/biome-text/documentation/tutorials/1-Training_a_text_classifier.html) shows you an example of how to configure a trainer. 

::: tip Tip

If you want to tune the trainer or use a cuda device, you can pass a `trainer = TrainerConfiguration(cuda_device=0, ...)` to the `Pipeline.train()` method. 
See the [TrainerConfiguration API](https://www.recogn.ai/biome-text/api/biome/text/configuration.html#trainerconfiguration) for a complete list of available configurations.

:::

Apart from the validation data source to estimate the generalization error, we will also pass in a test data set in case we want to do some Hyperparameter optimization and compare different encoder architectures in the end. 
For this we will create another `DataSource` pointing to our test data.

In [ ]:
test_ds = DataSource(source="https://biome-tutorials-data.s3-eu-west-1.amazonaws.com/token_classifier/test.json")

The training output will be saved in a folder specified by the `output` argument of the `train` method. 
It will contain the trained model weights and the metrics, as well as the vocabulary and a *log* folder for visualizing the training process with [tensorboard](https://www.tensorflow.org/tensorboard/).

When the training has finished it will automatically make a pass over the test data with the best weights to gather the test metrics.

In [ ]:
pl.train(
    output="output",
    training=train_ds,
    validation=valid_ds,
    test=test_ds,
)

The model above achieves an overall F1 score of around **0.95**, which is not bad when compared to [published results](https://nlpprogress.com/english/intent_detection_slot_filling.html) of the same data set.
You could continue the experiment changing the encoder to an LSTM network, try out a transformer architecture or fine tune the trainer.
But for now we will go on and make our first predictions with this trained model.

## Make your first predictions

Now that we trained our model we can go on to make our first predictions.
First we must load our trained model into a new `Pipeline`:

In [ ]:
pl_trained = Pipeline.from_pretrained("output/model.tar.gz")

We then provide the input expected by our `TaskHead` of the model to the `Pipeline.predict()` method.
In our case it is a `TokenClassification` head that classifies a `text` input. **Remember that the input has to be pre-tokenized!**

In [ ]:
text = "can you play biome text by backstreet recognais on Spotify".split()
prediction = pl_trained.predict(text)
list(zip(text, prediction["tags"][0]))

Apart from the most likely *tags*, the `prediction` dictionary contains the *logits* and *probs* of each of the label for each of the input token.